In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow. keras import layers, regularizers
import os
import pandas as pd
from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist. load_data()
x_train = x_train. reshape (-1, 28*28) .astype ("float32") / 255.0
x_test = x_test. reshape (-1, 28*28) .astype ("float32") / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
x_train.shape, y_train.shape

((60000, 784), (60000,))

In [19]:
class Dense (layers. Layer):
  def __init__ (self, units, input_dim):
    super(Dense, self) .__init__()
    self.w = self .add_weight(
        name='w', #it is very important for saving model, without naming weights models dont get saved
        shape=(input_dim,units),
        initializer='random_normal', #there are other methods too
        trainable=True,
    )

    self.b= self.add_weight(
        name='b',
        shape=(units,),
        initializer='zeros',
        trainable=True,
    )

  def call(self, input):
    return tf.matmul(input,self.w)+self.b



class Dense_new(layers. Layer):
  def __init__ (self, units):
    super(Dense_new, self) .__init__()
    self.units=units

  def build(self, input_shape): #here input_shape is given as parameter

    self.w = self .add_weight(
        name='w',
        shape=(input_shape[-1],self.units),
        initializer='random_normal',
        trainable=True,
    )

    self.b= self.add_weight(
        name='b',
        shape=(self.units,),
        initializer='zeros',
        trainable=True,
    )


  def call(self, input):
    return tf.matmul(input,self.w)+self.b

In [28]:
#we can create custom activation functions also
class Relu(layers.Layer):
  def __init__(self):
    super(Relu,self).__init__()

  def call(self, x):
    return tf.math.maximum(x,0)

In [29]:
class MyModel (keras.Model):

  def __init__ (self, num_classes=10) :
    super(MyModel, self).__init__()
    #self.dense1 = layers.Dense(64)
    #self.dense2 = layers.Dense(num_classes)
    #self.dense1=Dense(64,28*28) #we dont want to specify input dimension , sor for that we create build method and in that
                                #define weights
    #self.dense2=Dense(10,64)

    self.dense1=Dense_new(64)
    self.dense2=Dense_new(10)
    self.relu=Relu()

  def call(self, input_tensor):
    #x = tf.nn.relu(self.dense1(input_tensor))
    x= self.relu((self.dense1(input_tensor)))
    return self.dense2(x)

In [30]:
model=MyModel()

In [31]:
model.compile (optimizer=keras.optimizers.Adam(),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"],
)
model.fit(x_train, y_train, batch_size=64, epochs=3, verbose=2)
model.evaluate (x_test, y_test, batch_size=64, verbose=2)

Epoch 1/3
938/938 - 4s - loss: 0.4212 - accuracy: 0.8881 - 4s/epoch - 5ms/step
Epoch 2/3
938/938 - 2s - loss: 0.2097 - accuracy: 0.9407 - 2s/epoch - 2ms/step
Epoch 3/3
938/938 - 2s - loss: 0.1537 - accuracy: 0.9556 - 2s/epoch - 3ms/step
157/157 - 0s - loss: 0.1411 - accuracy: 0.9590 - 354ms/epoch - 2ms/step


[0.14110934734344482, 0.9589999914169312]

In [32]:
model.summary()

Model: "my_model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_new_6 (Dense_new)     multiple                  50240     
                                                                 
 dense_new_7 (Dense_new)     multiple                  650       
                                                                 
 relu (Relu)                 multiple                  0         
                                                                 
Total params: 50890 (198.79 KB)
Trainable params: 50890 (198.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
